# Our Task: Find similar company names

Now that we have completed our Python Primer, let's try to complete a real task, while at the same time trying to practice loops, iterations, and other Python functionality that we studied.

### Matching Company Names

Quite often, in our data, we have entries represented as strings that refer to the same entity but have different string representations (e.g., McDonald's vs McDonalds vs McDonald‎). We want to write code that will help in the task of finding such similar entries in our data.

Our task can be broken in the following tasks:
* **Step 1**: Read the data from a file into a list of strings in memory. We have two data sets under the `data` folder: The list of unique restaurant names from the NYC Restaurant Inspection dataset (`data/restaurant-names.txt`) and a list of bank names (`data/bank-names.txt`) We need to write Python code that will read these files and return a list of strings that are the company names.
* **Step 2**: We will then need to figure out how to compare two strings and find their similarity. For that, we need to write a function that takes as input two strings, and returns back their similarities (we will see below how to do that).
* **Step 3**: We will need to write a function that takes as input a company name, and returns back a list of matching company names, together with their similarity. Ideally, we would like to also sort the list based on the similarity (highest similarity metrics on top). As part of our learning process, we will use the _list comprehension_ approach to create the list. We will also use tuples as the elements of the list, so that the list contain elements such as `[("McDonalds", 0.88), ("McDonald's", 0.81),....]`.
* **Step 4**: In the final step, we will just perform the similarity computation across all companies in the dataset.


### STEP 1: Read the list of names from a file and create a list of names

In [1]:
# STEP 1: Read the list of names from a file and create a list of names
f = open("/data/restaurant-names.txt", "r")
content = f.read()
f.close()
lines = content.split("\n")

### STEP 2: Implement the similarity function

### Computing the similarity between two strings

There are many ways that we can calculate the similarity between two strings (e.g., see [1](http://en.wikipedia.org/wiki/String_metric), [2](http://en.wikipedia.org/wiki/Approximate_string_matching), [3](http://en.wikipedia.org/wiki/Plagiarism_detection), [4](https://www.cs.cmu.edu/~pradeepr/papers/ijcai03.pdf), [5](http://en.wikipedia.org/wiki/Category:String_similarity_measures), and links within). 

For our example, we will use the q-gram similarity metric. What is a q-gram? Simply a sequency of q-consecutive characters in the string. For example, the name "Panos" has the following 2-grams: "Pa", "an", "no", "os". (We can also add "#P" and "s#" if we want to capture the prefix and suffixes.) Strings that share a large number of q-grams are often similar. 

Hence, we can compute the similarity of two strings by first computing the set of q-grams for each string, and then compute the Jaccard coefficient between these sets. The [Jaccard coefficient](http://en.wikipedia.org/wiki/Jaccard_index) is simply defined as the size of the intersection divided by the size of the union of the two sets.

So, our task can be broken into two functions: 
* The first function takes a string and returns a list of the q-grams for the string, and 
* The second function takes as input two sets, and returns back their Jaccard coefficient.

In [2]:
# STEP 2: Now we implement our similarity function

# This returns a list of q-grams for a name
# getQgrams("Panos", 1) should return ["P", "a", "n", "s"]
# getQgrams("Panos", 2) should return ["Pa", "an", "no", "os"]
# etc
def getQgrams(name, q):
    # We add a little bit of padding of '#' characters
    # to create q-grams for the beginning and end of the 
    # string as well
    name = (q-1)*'#' + name + (q-1)*'#'
    length = len(name)
    qgrams = []
    for c in range(length):
        qgram = name[c:c+q]
        qgrams.append(qgram)
    return qgrams

# This function takes as input two names, computes their qgrams, and then computes
# the Jaccard coefficient (=intersection/union) of the two sets of qgrams
def jaccard(setA, setB):
    intersection = set(setA) & set(setB)
    union = set(setA) | set(setB)
    similarity = 1.0*len(intersection)/len(union)
    return similarity

def computeSimilarity(name1, name2, q):
    qgrams1 = getQgrams(name1.upper(), q)
    qgrams2 = getQgrams(name2.upper(), q)
    similarity = jaccard(qgrams1, qgrams2)
    return similarity

In [3]:
# Let's test our similarity function
computeSimilarity("Microsoft", "Micrsoft", 3)

0.6666666666666666

In [4]:
computeSimilarity("Microsoft", "Micrsoft", 2)

0.75

In [5]:
computeSimilarity("Microsoft", "Google", 3)

0.09523809523809523

### STEP 3: Compute similarity of a company against a list of company names

We now create a function that accepts a company name and a list of companies, and computes their similarity. This part will get us to exercise our for-loops, and also illustrate how we can use lists and tuples.

**Sorting a list of tuples**:_This part is a little bit advanced for now, so I will just give the code below. (Solution taken from http://stackoverflow.com/questions/3121979/how-to-sort-list-tuple-of-lists-tuples). Here is a small example below, which we will reuse in our function:_

In [6]:
data = [("Panos",0.5), ("Peter",0.6), ("Pan", 0.4)]
data.sort(key=lambda tupl: tupl[1], reverse=True)  # sorts in place, in descending order, based on the second element of the tuple
data

[('Peter', 0.6), ('Panos', 0.5), ('Pan', 0.4)]

In [7]:
# STEP 3: We now create a function that accepts a company name
# and a list of companies, and computes their similarity
# We have a threshold parameter (by default set to be 0.5)
# that restricts the results to only string pairs with similarity
# above the threshold

# queryData finds similar names to a query
def queryData(query, companyList, threshold, q):
    result = []
    for r in companyList:
        # I will compute the similarity of the "query" string
        # with each restaurant "r", where "r" is the variable in the for loop
        similarity = computeSimilarity(query, r, q)
        # if the similarity is above 0.5, I will print the result
        if similarity>threshold:
            result.append( (r, similarity) )
    return result

def companySimilarity(companyName, companyList, threshold = 0.5, q = 3):
    results = queryData(companyName, companyList, threshold, q)
    results.sort(key=lambda tupl: tupl[1], reverse=True)
    return results


In [8]:
# This is the name of the restaurant that I am looking for
query = "McDonalds"
# This is the similarity configuration parameter
q = 3
threshold = 0.25
# lines contains the list of companiues
companySimilarity(query, lines, threshold, q)

[('MCDONALDS', 1.0),
 ('MC DONALDS', 0.6875),
 ("MCDONALD'S", 0.6875),
 ('MCDONALD', 0.6666666666666666),
 ("MCDONALD'S ", 0.5555555555555556),
 ('MCDONALDS 17754', 0.5238095238095238),
 ("MC DONALD'S", 0.47368421052631576),
 ('MCDONALDS (#11542)', 0.4583333333333333),
 ('MCDONALDS RESTAURANT', 0.4230769230769231),
 ('MCDONALD AVENUE DINER', 0.35714285714285715),
 ("MCDONALD'S RESTAURANT", 0.35714285714285715),
 ("MCDONALD'S (MACY'S 7TH FL)", 0.3125),
 ('MCDONALDS FAST FOOD RESTAURANT', 0.3055555555555556),
 ("MCDONALD'S RESTAURANT #3391", 0.29411764705882354),
 ("MONA'S", 0.2777777777777778),
 ('MRS. FIELDS', 0.2727272727272727)]

### Step 4: Perform the similarity computation across all companies in the dataset.

In [ ]:
# STEP 4: We are almost done. We now just go through all the companies in the list
# and we call the companySimilarity function that computes the similar company names
# for all the companies in the list. We store the results in a dictionary, with the 
# key being the company name, and the value being a "list of tuples" with the 
# similar company names and the corresponding similarity value.
i = 0
for companyName in lines:
    matches = companySimilarity(companyName, lines, threshold, q)
    print(companyName, "===>")
    # we will print the top-4 matching names
    print(matches[:4])
    print("================")
    i += 1
    if i>5:
        break